## CSV to midi

### Setup

In [1]:
import pandas as pd
import numpy as np
import pretty_midi
import fluidsynth
from IPython import display

In [2]:
bpm = 90
spb = 60/bpm  # seconds per beat
_SAMPLING_RATE = 16000

In [3]:
df = pd.read_csv ('data/mozart piano sonatas - basic ideas - K547a-II.csv')
df.head()

,note_name,duration,measure,beat,subdiv,harm,from_root,NCT,BI2,CI,trill
0,c6,8,0,2,1,I,5,0,0,0,0
1,c6,8,0,2,3,I,5,0,0,0,0
2,a5,8,1,1,1,vi,5,0,0,0,0
3,r,8,1,1,3,0,0,0,0,0,0
4,bb5,8,1,2,1,ii,3,0,0,0,0


### Process the CSV

In [4]:
beat_dict_simple = {
    '1': spb * 4, 
    '2': spb * 2, 
    '4': spb, 
    '8': spb / 2.0,
    '16': spb / 4.0,
    '32': spb / 8.0,
}

beat_dict_compound = {
    '1': spb * (8/3.0),
    '2': spb * (4/3.0),
    '4': spb * (2/3.0),
    '8': spb / 3.0,
    '16': spb / 6.0,
}

In [5]:
def simple_duration_to_seconds(x):
    if type(x) != 'str':
        x = str(x)
    if x[-1] == 'd':
        s = beat_dict_simple[x[:-1]] * 1.5
    else:
        s = beat_dict_simple[x]
    return s

def compound_duration_to_seconds(x):
    if type(x) != 'str':
        x = str(x)
    if x[-1] == 'd':
        s = beat_dict_compound[x[:-1]] * 1.5
    else:
        s = beat_dict_compound[x]
    return s
    

In [6]:
def notes_to_midi(
    notes: pd.DataFrame,
    out_file: str,
    instrument_name: str,
    velocity: int = 100 # note loudness
) -> pretty_midi.PrettyMIDI:
    
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(
        program=pretty_midi.instrument_name_to_program(
            instrument_name))
    
    if notes['subdiv'].max() > 4:
        notes['dur(s)'] = notes.duration.apply(compound_duration_to_seconds)
    else:
        notes['dur(s)'] = notes.duration.apply(simple_duration_to_seconds)
    
    prev_start = 0
    for i, note in notes.iterrows():
        start = float(prev_start)
        end = float(prev_start + note['dur(s)'])
        if note['note_name'] == 'r':
            start += float(note['dur(s)'])
            prev_start = start
        else:
            note = pretty_midi.Note(
                velocity=velocity,
                pitch=int(pretty_midi.note_name_to_number(note['note_name'])),
                start=start,
                end=end
            )
            instrument.notes.append(note)
            prev_start = end
            
    pm.instruments.append(instrument)
    return pm

### Playback the midi

In [7]:
def display_audio(pm: pretty_midi.PrettyMIDI):
    waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
    return display.Audio(waveform, rate=_SAMPLING_RATE)

In [8]:
# instrument_name = 'Electric Piano 1'
instrument_name = 'Acoustic Guitar (nylon)'
out_file = 'example.midi'

example_pm = notes_to_midi(df, out_file, instrument_name)

In [9]:
display_audio(example_pm)

fluidsynth: error: Unknown integer parameter 'synth.sample-rate'


### Useful code

In [ ]:
# df['dur(s)'] = df.duration.apply(compound_duration_to_seconds)

In [ ]:
# prev_start = 0
# start = []
# end = []
# for _ in df['dur(s)']:
#     start.append(prev_start)
#     end_time = prev_start + _
#     end.append(end_time)
#     prev_start = end_time

In [ ]:
# df['start'] = pd.Series(start)
# df['end'] = pd.Series(end)
# df['pitch'] = df['note_name'].apply(lambda x: pretty_midi.note_name_to_number(x))